# Cross-Model Validation: Gemma-2B Layer Analysis

**Purpose:** Test if phase-structured embedding-output relationship exists in SFT models.

**Research Question:** Does Gemma-2B (SFT, not RLHF) show:
1. The same late-layer inversion as Pythia/Llama?
2. Different phase structure due to SFT training?

**Context:** Original paper showed r≈+0.95 at category level (confounded by GT-Numeric).
Layer analysis will reveal the true pattern.

**Expected Runtime:** ~1.5h on A100

---

**Author:** Davide D'Elia  
**Date:** 2026-01-03  
**Model:** google/gemma-2b

## 1. Setup

In [ ]:
!pip install -q transformers accelerate torch numpy scipy matplotlib scikit-learn huggingface_hub

In [ ]:
import json
import warnings
from datetime import datetime
from typing import Dict, List, Tuple

import numpy as np
import torch
import matplotlib.pyplot as plt
from scipy import stats
from transformers import AutoModelForCausalLM, AutoTokenizer

warnings.filterwarnings('ignore')

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

N_BOOTSTRAP = 10000
CI_LEVEL = 0.95

print(f'PyTorch: {torch.__version__}')
print(f'CUDA: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

In [ ]:
# ========================================
# HUGGINGFACE LOGIN
# ========================================
# Gemma requires accepting terms at: https://huggingface.co/google/gemma-2b

try:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
    print('Token aus Colab Secrets geladen')
except:
    HF_TOKEN = None

if not HF_TOKEN:
    HF_TOKEN = ''  # <-- Token hier einfuegen

if not HF_TOKEN:
    from huggingface_hub import notebook_login
    notebook_login()
else:
    from huggingface_hub import login
    login(token=HF_TOKEN)
    print('Eingeloggt')

## 2. Load Model

In [ ]:
MODEL_NAME = 'google/gemma-2b'
MODEL_DISPLAY = 'Gemma-2B'

print(f'Loading {MODEL_DISPLAY}...')
print('NOTE: SFT model (Supervised Fine-Tuning, not RLHF)')

token_arg = HF_TOKEN if HF_TOKEN else True

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, token=token_arg)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    token=token_arg,
    torch_dtype=torch.float16,
    device_map='auto',
    output_hidden_states=True
)

print(f'Model loaded')
print(f'Layers: {model.config.num_hidden_layers}')
print(f'Hidden size: {model.config.hidden_size}')

## 3. Load Dataset

In [ ]:
!wget -q https://raw.githubusercontent.com/buk81/uniformity-asymmetry/main/dataset.json

with open('dataset.json', 'r') as f:
    DATASET = json.load(f)

ALL_PAIRS = []
for cat_name, cat_data in DATASET.items():
    for pair in cat_data['pairs']:
        ALL_PAIRS.append({'stmt_a': pair[0], 'stmt_b': pair[1], 'category': cat_name})

print(f'Categories: {list(DATASET.keys())}')
print(f'Total pairs: {len(ALL_PAIRS)}')

## 4. Core Functions

In [ ]:
def get_layer_embedding(text, model, tokenizer, layer_idx):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512).to(model.device)
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    layer_hidden = outputs.hidden_states[layer_idx]
    return layer_hidden[0, 1:, :].mean(dim=0).cpu().numpy().astype(np.float32)

def get_output_preference(text_a, text_b, model, tokenizer):
    def get_nll(text):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512).to(model.device)
        with torch.no_grad():
            outputs = model(**inputs, labels=inputs['input_ids'])
        return outputs.loss.item()
    return get_nll(text_b) - get_nll(text_a)

def cosine_similarity(a, b):
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b) + 1e-10))

def bootstrap_correlation(x, y, n_bootstrap=10000, ci_level=0.95):
    n = len(x)
    r_obs, p = stats.pearsonr(x, y)
    rs = []
    for _ in range(n_bootstrap):
        idx = np.random.choice(n, size=n, replace=True)
        if np.std(x[idx]) > 0 and np.std(y[idx]) > 0:
            r, _ = stats.pearsonr(x[idx], y[idx])
            rs.append(r)
    rs = np.array(rs)
    alpha = 1 - ci_level
    return float(r_obs), float(np.percentile(rs, alpha/2*100)), float(np.percentile(rs, (1-alpha/2)*100)), float(p)

print('Functions defined.')

## 5. Collect Embeddings

In [ ]:
N_LAYERS = model.config.num_hidden_layers
# Gemma-2B has 18 layers, sample every 3rd
LAYERS_TO_TEST = list(range(0, N_LAYERS + 1, max(1, N_LAYERS // 8)))
if N_LAYERS not in LAYERS_TO_TEST:
    LAYERS_TO_TEST.append(N_LAYERS)

print(f'Testing layers: {LAYERS_TO_TEST}')
print(f'Collecting embeddings for {len(ALL_PAIRS)} pairs...')

pair_data = []
start = datetime.now()

for i, pair in enumerate(ALL_PAIRS):
    if (i + 1) % 25 == 0:
        elapsed = (datetime.now() - start).total_seconds() / 60
        print(f'  [{i+1:03d}/{len(ALL_PAIRS)}] - {elapsed:.1f} min')
    
    pref = get_output_preference(pair['stmt_a'], pair['stmt_b'], model, tokenizer)
    layer_embs = {}
    for l in LAYERS_TO_TEST:
        emb_a = get_layer_embedding(pair['stmt_a'], model, tokenizer, l)
        emb_b = get_layer_embedding(pair['stmt_b'], model, tokenizer, l)
        layer_embs[l] = {'a': emb_a, 'b': emb_b}
    pair_data.append({'pref': pref, 'cat': pair['category'], 'embs': layer_embs})

print(f'Done in {(datetime.now() - start).total_seconds() / 60:.1f} min')

## 6. Pair-Level Analysis

In [ ]:
def compute_centroid_asymmetry(pair_data, layer_idx):
    embs_a = np.array([p['embs'][layer_idx]['a'] for p in pair_data])
    embs_b = np.array([p['embs'][layer_idx]['b'] for p in pair_data])
    cent_a, cent_b = embs_a.mean(0), embs_b.mean(0)
    dist_a = np.array([cosine_similarity(e, cent_a) for e in embs_a])
    dist_b = np.array([cosine_similarity(e, cent_b) for e in embs_b])
    return dist_a - dist_b

all_prefs = np.array([p['pref'] for p in pair_data])

print('=' * 70)
print(f' {MODEL_DISPLAY}: PAIR-LEVEL ANALYSIS (n={len(pair_data)})')
print('=' * 70)

results = {}
for l in LAYERS_TO_TEST:
    metric = compute_centroid_asymmetry(pair_data, l)
    r, ci_lo, ci_hi, p = bootstrap_correlation(metric, all_prefs, N_BOOTSTRAP, CI_LEVEL)
    sig = '***' if not (ci_lo <= 0 <= ci_hi) else ''
    results[l] = {'r': r, 'ci_lo': ci_lo, 'ci_hi': ci_hi, 'sig': not (ci_lo <= 0 <= ci_hi)}
    print(f'Layer {l:2d}: r={r:+.3f} CI=[{ci_lo:+.3f},{ci_hi:+.3f}] {sig}')

## 7. Phase Structure

In [ ]:
# Determine phase boundaries based on layer count
n_layers = len(LAYERS_TO_TEST)
early_layers = LAYERS_TO_TEST[:n_layers//3]
mid_layers = LAYERS_TO_TEST[n_layers//3:2*n_layers//3]
late_layers = LAYERS_TO_TEST[2*n_layers//3:]

early_mean = np.mean([results[l]['r'] for l in early_layers])
mid_mean = np.mean([results[l]['r'] for l in mid_layers])
late_mean = np.mean([results[l]['r'] for l in late_layers])

print(f'\nPhase Structure:')
print(f'  Early {early_layers}: mean r = {early_mean:+.3f}')
print(f'  Mid   {mid_layers}: mean r = {mid_mean:+.3f}')
print(f'  Late  {late_layers}: mean r = {late_mean:+.3f}')

# Determine pattern
if early_mean > 0.1 and late_mean < -0.1:
    pattern = 'PYTHIA_PATTERN: positive early, negative late'
elif late_mean < -0.1:
    pattern = 'LATE_INVERSION: negative in late layers'
elif abs(early_mean) < 0.15 and abs(mid_mean) < 0.15 and abs(late_mean) < 0.15:
    pattern = 'DECOUPLED: weak correlations throughout'
else:
    pattern = 'OTHER'

print(f'\n>>> Pattern: {pattern} <<<')

## 8. Comparison with Other Models

In [ ]:
# Reference values from Pythia and Llama
print('=' * 70)
print(' CROSS-MODEL COMPARISON')
print('=' * 70)

print(f'\nPhase Means Comparison:')
print(f'{"Model":<25} {"Early":>10} {"Mid":>10} {"Late":>10}')
print('-' * 55)
print(f'{"Pythia-6.9B":<25} {+0.44:>+10.2f} {+0.36:>+10.2f} {-0.17:>+10.2f}')
print(f'{"Llama-3.1-8B Base":<25} {+0.05:>+10.2f} {-0.16:>+10.2f} {-0.30:>+10.2f}')
print(f'{"Llama-3.1 Instruct+Templ":<25} {-0.47:>+10.2f} {-0.58:>+10.2f} {-0.60:>+10.2f}')
print(f'{MODEL_DISPLAY:<25} {early_mean:>+10.2f} {mid_mean:>+10.2f} {late_mean:>+10.2f}')

# Check late layer inversion
final_layer = LAYERS_TO_TEST[-1]
final_r = results[final_layer]['r']
print(f'\nFinal layer ({final_layer}) correlation: r = {final_r:+.3f}')
if final_r < -0.1 and results[final_layer]['sig']:
    print('>>> LATE-LAYER INVERSION CONFIRMED <<<')
elif final_r < 0:
    print('>>> Negative trend in late layer (not significant) <<<')
else:
    print('>>> NO late-layer inversion <<<')

## 9. Visualization

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

layers = list(results.keys())
rs = [results[l]['r'] for l in layers]
ci_los = [results[l]['ci_lo'] for l in layers]
ci_his = [results[l]['ci_hi'] for l in layers]

yerr_lo = [r - ci_lo for r, ci_lo in zip(rs, ci_los)]
yerr_hi = [ci_hi - r for r, ci_hi in zip(rs, ci_his)]

ax.errorbar(layers, rs, yerr=[yerr_lo, yerr_hi], fmt='o-', capsize=5, 
            capthick=2, markersize=8, color='green', label=MODEL_DISPLAY)

# Mark significant
for l, r in zip(layers, rs):
    if results[l]['sig']:
        ax.scatter([l], [r], color='red', s=150, zorder=5, marker='*')

ax.axhline(0, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('Layer', fontsize=12)
ax.set_ylabel('r(centroid_asymmetry, output)', fontsize=12)
ax.set_title(f'{MODEL_DISPLAY}: Layer-wise Correlation\nPattern: {pattern}', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('gemma_layer_analysis.png', dpi=150)
plt.show()
print('Plot saved')

## 10. Save Results

In [ ]:
save_data = {
    'timestamp': datetime.now().isoformat(),
    'model': MODEL_NAME,
    'model_display': MODEL_DISPLAY,
    'model_type': 'SFT (Supervised Fine-Tuning)',
    'n_layers': N_LAYERS,
    'layers_tested': LAYERS_TO_TEST,
    'n_pairs': len(pair_data),
    'n_bootstrap': N_BOOTSTRAP,
    'results': {str(k): v for k, v in results.items()},
    'phase_structure': {
        'early_mean': float(early_mean),
        'mid_mean': float(mid_mean),
        'late_mean': float(late_mean),
        'pattern': pattern
    }
}

fname = f'gemma_cross_validation_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json'
with open(fname, 'w') as f:
    json.dump(save_data, f, indent=2)
print(f'Saved: {fname}')

from google.colab import files
files.download(fname)
files.download('gemma_layer_analysis.png')